<a href="https://colab.research.google.com/github/MehrdadJalali-AI/InverseLinkPredcition/blob/main/ILP_AllThreshold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
import networkx.algorithms.community as nx_comm
import time
import warnings

# Suppress RDKit warnings
from rdkit import rdBase
rdBase.DisableLog("rdApp.*")
warnings.filterwarnings("ignore")

def load_edges_list(filename):
    """Loads edge list from a CSV file. O(m)"""
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        raise FileNotFoundError(f"Edge list file '{filename}' not found.")

def generate_fingerprint(smiles):
    """Generates a molecular fingerprint from a SMILES string. O(1) per call"""
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return np.zeros(1024, dtype=np.float32)
        return np.array(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024), dtype=np.float32)
    except Exception as e:
        print(f"SMILES Parse Error: {e}")
        return np.zeros(1024, dtype=np.float32)

def label_encode_metal_names(metal_names):
    """Encodes metal names as integers. O(n)"""
    unique_metals = np.unique(metal_names)
    metal_dict = {metal: idx for idx, metal in enumerate(unique_metals)}
    return np.array([metal_dict[metal] for metal in metal_names], dtype=np.int32)

def load_summary_data(filename, node_labels):
    """Loads and preprocesses summary data to extract features. O(n * f)"""
    try:
        summary_data = pd.read_csv(filename, index_col=0)
        linker_smiles = summary_data['linker SMILES']
        linker_features = np.stack(linker_smiles.apply(generate_fingerprint).values)
        metal_features = label_encode_metal_names(summary_data['metal']).reshape(-1, 1)
        other_features = summary_data[['Largest Cavity Diameter', 'Pore Limiting Diameter',
                                    'Largest Free Sphere']].astype(np.float32).values
        features = np.concatenate((linker_features, metal_features, other_features), axis=1)

        if (len(summary_data) == len(node_labels) and
            all(label in summary_data.index for label in node_labels)):
            return pd.DataFrame(features, index=node_labels)
        else:
            print("Warning: Mismatch between summary data and node labels.")
            return pd.DataFrame(features, index=summary_data.index)
    except FileNotFoundError:
        raise FileNotFoundError(f"Summary data file '{filename}' not found.")

def build_and_compile_model(input_shape):
    """Builds and compiles a neural network model. O(1) (no training here)"""
    model = models.Sequential([
        layers.Input(shape=(input_shape,)),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

def predict_link_importance(model, node_features):
    """Predicts link importance using a trained model. O(n * f * h)"""
    return model.predict(node_features, verbose=0).flatten()

def optimize_combination_for_modularity(graph, features, importance_scores, initial_weights, threshold=0.2):
    """Optimizes combination for modularity. O(m * log(n))"""
    best_modularity, best_a = -1, 0.0
    a_range = np.linspace(0, 1, 11)

    for a in a_range:  # 11 iterations, O(1)
        combined_scores = a * importance_scores + (1 - a) * initial_weights  # O(m)
        edges_to_remove = [edge for edge, score in zip(graph.edges(), combined_scores) if score < threshold]  # O(m)
        graph_copy = graph.copy()  # O(m + n)
        graph_copy.remove_edges_from(edges_to_remove)  # O(m)
        communities = nx_comm.greedy_modularity_communities(graph_copy)  # O(m * log(n))
        modularity = nx_comm.modularity(graph_copy, communities)  # O(m + n)
        if modularity > best_modularity:
            best_modularity, best_a = modularity, a

    print(f"Best combination coefficient (a): {best_a} with modularity: {best_modularity}")
    return best_a

def remove_edges_and_nodes(graph, importance_scores, initial_weights, best_a, threshold):
    """Removes edges and nodes based on combined scores. O(m + n)"""
    combined_scores = best_a * importance_scores + (1 - best_a) * initial_weights  # O(m)
    edges_to_remove = [(u, v) for (u, v, _), score in zip(graph.edges(data=True), combined_scores) if score < threshold]  # O(m)
    graph.remove_edges_from(edges_to_remove)  # O(m)
    graph.remove_nodes_from(list(nx.isolates(graph)))  # O(n)
    return graph
def save_remaining_node_features(graph, summary_data, filename):
    """Saves features of remaining nodes to a file."""
    remaining_nodes = list(graph.nodes())
    remaining_features = summary_data.loc[remaining_nodes]
    remaining_features.to_csv(filename, index=False)
    print(f"Features of remaining nodes saved to '{filename}'.")

def calculate_metrics(original_graph, sparsified_graph, computation_time):
    """Calculates metrics for the sparsified graph. O(m * log(n))"""
    orig_nodes, orig_edges = original_graph.number_of_nodes(), original_graph.number_of_edges()  # O(1)
    new_nodes, new_edges = sparsified_graph.number_of_nodes(), sparsified_graph.number_of_edges()  # O(1)

    node_reduction = ((orig_nodes - new_nodes) / orig_nodes) * 100 if orig_nodes > 0 else 0  # O(1)
    edge_reduction = ((orig_edges - new_edges) / orig_edges) * 100 if orig_edges > 0 else 0  # O(1)
    avg_degree = sum(d for _, d in sparsified_graph.degree()) / new_nodes if new_nodes > 0 else 0  # O(n)
    communities = nx_comm.greedy_modularity_communities(sparsified_graph)  # O(m * log(n))
    modularity = nx_comm.modularity(sparsified_graph, communities) if new_edges > 0 else 0  # O(m + n)

    return {
        'Threshold': None,
        'Remaining Nodes': new_nodes,
        'Remaining Edges': new_edges,
        'Node Reduction (%)': node_reduction,
        'Edge Reduction (%)': edge_reduction,
        'Average Degree': avg_degree,
        'Modularity': modularity,
        'Communities': len(communities),
        'Computation Time (s)': computation_time
    }

def main():
    """Main execution function. O(T * m * log(n)) overall"""
    start_time = time.time()
    config = {
        'edges_list_filename': 'edges_list_0.8_Full_2.csv',
        'summary_data_filename': '1M1L3D_summary.csv',
        'output_base': 'sparsified_graph_edges',
        'features_base': 'remaining_node_features',
        'thresholds': [0.5, 0.8, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]
    }

    # Load and prepare data: O(m + n * f)
    edges_list = load_edges_list(config['edges_list_filename'])
    node_labels = pd.concat([edges_list['source'], edges_list['target']]).unique()
    summary_data = load_summary_data(config['summary_data_filename'], node_labels)

    # Build original graph: O(m)
    original_graph = nx.Graph()
    for _, row in edges_list.iterrows():
        original_graph.add_edge(row['source'], row['target'], weight=row['weight'])
    print(f"Original graph: {original_graph.number_of_edges()} edges, {original_graph.number_of_nodes()} nodes")

    # Prepare features: O(n * f)
    node_features = np.array([summary_data.loc[node].values for node in original_graph.nodes()])
    scaler = StandardScaler()
    node_features_scaled = scaler.fit_transform(node_features)

    # Build model (training placeholder): O(1) currently
    model = build_and_compile_model(node_features_scaled.shape[1])
    # Hypothetical training: O(e * n * f * h) if implemented

    # Generate dummy scores (edge weight calc): O(m) currently, O(n * f * h + m) with model
    importance_scores = np.random.rand(len(original_graph.edges()))
    initial_weights = np.array([data['weight'] for _, _, data in original_graph.edges(data=True)])

    # Optimize combination: O(m * log(n))
    best_a = optimize_combination_for_modularity(original_graph, summary_data, importance_scores, initial_weights)

    # Results storage
    results = []

    # Iterate over thresholds: O(T * (m + n + m * log(n)))
    for threshold in config['thresholds']:
        threshold_start_time = time.time()

        graph = original_graph.copy()  # O(m + n)
        graph = remove_edges_and_nodes(graph, importance_scores, initial_weights, best_a, threshold)  # O(m + n)
        computation_time = time.time() - threshold_start_time

        metrics = calculate_metrics(original_graph, graph, computation_time)  # O(m * log(n))
        metrics['Threshold'] = threshold
        results.append(metrics)

        output_edges_filename = f"{config['output_base']}_{threshold:.2f}.csv"
        output_features_filename = f"{config['features_base']}_{threshold:.2f}.csv"
        nx.write_edgelist(graph, output_edges_filename, data=['weight'])  # O(m)
        save_remaining_node_features(graph, summary_data, output_features_filename)  # O(n)

        print(f"\nThreshold: {threshold:.2f}")
        for key, value in metrics.items():
            if isinstance(value, float):
                print(f"{key}: {value:.4f}")
            else:
                print(f"{key}: {value}")

    # Save results: O(T)
    results_df = pd.DataFrame(results)
    results_df = results_df[['Threshold', 'Remaining Nodes', 'Remaining Edges',
                            'Node Reduction (%)', 'Edge Reduction (%)',
                            'Average Degree', 'Modularity', 'Communities',
                            'Computation Time (s)']]
    results_df.to_csv('sparsification_results.csv', index=False)
    print(f"\nAll results saved to 'sparsification_results.csv'")
    print(f"Total runtime: {time.time() - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

Original graph: 414650 edges, 12561 nodes
Best combination coefficient (a): 0.8 with modularity: 0.8328619530154958
Features of remaining nodes saved to 'remaining_node_features_0.50.csv'.

Threshold: 0.50
Threshold: 0.5000
Remaining Nodes: 12022
Remaining Edges: 245602
Node Reduction (%): 4.2911
Edge Reduction (%): 40.7688
Average Degree: 40.8588
Modularity: 0.8336
Communities: 770
Computation Time (s): 1.1601
Features of remaining nodes saved to 'remaining_node_features_0.80.csv'.

Threshold: 0.80
Threshold: 0.8000
Remaining Nodes: 10548
Remaining Edges: 89872
Node Reduction (%): 16.0258
Edge Reduction (%): 78.3258
Average Degree: 17.0406
Modularity: 0.8357
Communities: 694
Computation Time (s): 0.9631
Features of remaining nodes saved to 'remaining_node_features_0.90.csv'.

Threshold: 0.90
Threshold: 0.9000
Remaining Nodes: 8938
Remaining Edges: 38243
Node Reduction (%): 28.8432
Edge Reduction (%): 90.7770
Average Degree: 8.5574
Modularity: 0.8466
Communities: 687
Computation Time (